# Pandas Run

In [1]:
import os
import pandas as pd
import time
import numpy as np
import shutil
import gc
import joblib
import psutil

In [3]:
# PATH_DATA =  '../../data-testing/'
PATH_DATA = 'O:/df-showdown-data/'
TEMP_PATH = PATH_DATA+'temp/'
PATH_OUTPUT = '../output3/'

SLA_NAME = 'pandas-sla.pkl'
MEM_NAME = 'pandas-mem.pkl'


In [4]:
all_sla = {}
all_memory = {}

In [5]:
def get_memory():
    est_memory = psutil.virtual_memory()[3]/1000000000
    return est_memory

In [6]:
# all_sla = joblib.load(PATH_OUTPUT+SLA_NAME)
# all_memory = joblib.load(PATH_OUTPUT+MEM_NAME)

##     Task1 & Task2- I/O

In [7]:
all_sla['task1'] = {}
all_sla['task1']['csv'] = {}
all_sla['task1']['prq'] = {}

all_sla['task2'] = {}
all_sla['task2']['csv'] = {}
all_sla['task2']['prq'] = {}



all_memory['task1'] = {}
all_memory['task1']['csv'] = {}
all_memory['task1']['prq'] = {}

all_memory['task2'] = {}
all_memory['task2']['csv'] = {}
all_memory['task2']['prq'] = {}


In [11]:
for i in range(4):
    print(i)
    if i<=5:
        fn = 'data_{}.csv'.format(i)
        start_mem = get_memory()
        start = time.perf_counter()
        dtemp1 = pd.read_csv(PATH_DATA+fn)
        end = time.perf_counter() - start
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        nrows = str(int(dtemp1.shape[0]/1000000))
        all_sla['task1']['csv'][nrows]=end
        all_memory['task1']['csv'][nrows]=end_mem

        start_mem = get_memory()
        start = time.perf_counter()
        dtemp1.to_csv(TEMP_PATH + 'temp.csv',index=False)
        end = time.perf_counter() - start
        end_mem = get_memory() - start_mem
        all_sla['task2']['csv'][nrows]=end
        all_memory['task2']['csv'][nrows]=end_mem

        del dtemp1
        time.sleep(3)
        gc.collect()
        os.remove(TEMP_PATH + 'temp.csv')
        time.sleep(2)

    fn = 'data_{}.parquet'.format(i)
    start_mem = get_memory()
    start = time.perf_counter()
    dtemp1 = pd.read_parquet(PATH_DATA+fn, engine='pyarrow')
    end = time.perf_counter() - start
    end_mem = get_memory() - start_mem
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['prq'][nrows]=end
    all_memory['task1']['prq'][nrows]=end_mem
    
    
    start_mem = get_memory()
    start = time.perf_counter()
    dtemp1.to_parquet(TEMP_PATH + 'temp.parquet',index=False)
    end = time.perf_counter() - start
    end_mem = get_memory() - start_mem
    all_sla['task2']['prq'][nrows]=end
    all_memory['task2']['prq'][nrows]=end_mem

    del dtemp1
    time.sleep(2)
    gc.collect()
    os.remove(TEMP_PATH + 'temp.parquet')

0
1
2
3


In [14]:
all_sla

{'task1': {'csv': {'0': 0.3123042999999939,
   '1': 1.6353573999999753,
   '5': 6.135828000000004,
   '10': 12.172257300000012},
  'prq': {'0': 0.0827970999999934,
   '1': 0.49277770000000487,
   '5': 2.114694799999995,
   '10': 4.113156500000002}},
 'task2': {'csv': {'0': 0.7298135000000059,
   '1': 5.06882809999999,
   '5': 20.082997499999976,
   '10': 40.30023490000002},
  'prq': {'0': 0.14882910000000038,
   '1': 1.1431090000000097,
   '5': 4.1271544999999605,
   '10': 7.176750799999979}}}

In [15]:
all_memory

{'task1': {'csv': {'0': 0.04176281599999854,
   '1': 0.15261286399999996,
   '5': 0.5899796479999999,
   '10': 1.152356352},
  'prq': {'0': 0.006414336000000631,
   '1': 0.04135731200000059,
   '5': 0.2671656959999993,
   '10': 0.6080839680000008}},
 'task2': {'csv': {'0': -0.0017571840000005778,
   '1': -0.042983423999999104,
   '5': -0.01724825600000024,
   '10': -0.01624064000000125},
  'prq': {'0': -0.0010854399999988829,
   '1': 0.05018419199999968,
   '5': 0.023990271999998924,
   '10': 0.09826304000000086}}}

In [10]:
# del dtemp1
# gc.collect()

## Get Data for other task

In [5]:
all_df = [pd.read_parquet(PATH_DATA+'data_{}.parquet'.format(i)) for i in [2,3,4,5,6,7]]
# all_df = [pd.read_parquet(PATH_DATA+'data_{}.parquet'.format(i)) for i in [1,2,3]]
df_right = pd.read_parquet(PATH_DATA+'data_to_join.parquet')

In [6]:
len(all_df)

3

In [7]:
df_right

,Date,ship-service-level,amount_avg
0,03-31-22,Expedited,629.237410
1,03-31-22,Standard,618.254348
2,04-01-22,Expedited,656.542718
3,04-01-22,Standard,569.998795
4,04-02-22,Expedited,643.696460
...,...,...,...
177,06-27-22,Standard,647.401391
178,06-28-22,Expedited,680.520651
179,06-28-22,Standard,681.640388
180,06-29-22,Expedited,656.470309


## Task3 - Sorting

In [8]:
n_task = 'task3'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [9]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.sort_values(['Date','Amount'],ascending=[True,False])
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()


1.03
5.03
10.06


In [10]:
all_sla[n_task]

{'1': 0.4081974000000059, '5': 1.3883672000000047, '10': 2.7218924999999956}

In [11]:
all_memory[n_task]

{'1': 0.06862028799999997, '5': 0.28434022399999925, '10': 0.5636464639999996}

## Task4 - Filtering

In [18]:
n_task = 'task4'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [19]:
# dtemp1 = all_df[0]
filt1 = ['Shipped - Delivered to Buyer', 'Cancelled']
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1[(dtemp1['Amount']>300) & (~dtemp1['Status'].isin(filt1))]
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()
    


25.02
50.04
75.06
100.08
130.01
175.02


In [20]:
all_sla[n_task]

{'25': 1.8878453270008322,
 '50': 3.5764362230002007,
 '75': 5.402434069999799,
 '100': 7.140304140000808,
 '130': 9.396109327000886,
 '175': 12.683115039999393}

In [21]:
all_memory[n_task]

{'25': 0.7036968959999967,
 '50': 1.4724546560000036,
 '75': 2.2107013119999976,
 '100': 2.947256320000001,
 '130': 3.827990528000001,
 '175': 5.154140160000004}

## Task5 - Merging

In [12]:
n_task = 'task5'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [13]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = pd.merge(dtemp1,df_right,on=['Date','ship-service-level'],how='left')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))

    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

1.03
5.03
10.06


In [14]:
all_sla[n_task]

{'1': 0.3041627000000062, '5': 1.1288678999999888, '10': 2.470291599999996}

In [15]:
all_memory[n_task]

{'1': 0.04614963200000055, '5': 0.2403328000000009, '10': 0.5011742720000001}

## Task6 - udf apply

In [ ]:
n_task = 'task6'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [ ]:
def fun(x):
    x0 = x + 1
    for i in range(25):
        if x0<800:
            x0 += i
            x0 = (x0/3.0)*2.5
            x0 = x0*1.2
        else:
            x0 += i/2.0
            x0 = (x0/4.0)*3.8

    return x0

In [28]:
# dtemp1=all_df[0]
# dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem
    
    del dtemp1
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

130.01
175.02


In [29]:
all_sla[n_task]

{'25': 102.20912465699985,
 '50': 204.11441018799997,
 '75': 349.69875541000147,
 '100': 489.2295786049999,
 '130': 547.7400657990001,
 '175': 720.9182632510001}

In [30]:
all_memory[n_task]

{'25': 0.2011545599999991,
 '50': 0.4011786240000035,
 '75': 24.605212672,
 '100': -27.098406912,
 '130': 1.0493460479999968,
 '175': 1.4058332159999978}

## Task7 - aggregation

In [31]:
n_task = 'task7'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [32]:
def p25(x):
    return np.percentile(x,25)

def p75(x):
    return np.percentile(x,75)

In [33]:
# dtemp1=all_df[0]
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':[np.mean, np.size, p25, p75]})
    dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':'mean'})

    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02
50.04
75.06
100.08
130.01
175.02


In [34]:
all_sla[n_task]

{'25': 4.4222110110004,
 '50': 8.711333011000534,
 '75': 13.207925713000805,
 '100': 18.71505420200083,
 '130': 23.023890553000456,
 '175': 30.91064094000103}

In [35]:
all_memory[n_task]

{'25': -0.0010854399999971065,
 '50': 0.000671744000001695,
 '75': -0.00011878400000142619,
 '100': 0.0005283840000060991,
 '130': -0.024449024000006148,
 '175': 0.0028303359999952704}

## Save results

In [36]:
joblib.dump(all_sla,PATH_OUTPUT+SLA_NAME)
joblib.dump(all_memory,PATH_OUTPUT+MEM_NAME)

['../output2/pandas-mem.pkl']